## How to use mmdetection
I've already created train images and also coco annotations.

- Create image dataset  
notebook: https://www.kaggle.com/bamps53/create-image-dataset  
dataset: https://www.kaggle.com/bamps53/nfl2021-train-images  

- Convert ground truth to coco format(with train/val split)  
notebook: https://www.kaggle.com/bamps53/create-coco-format-annotations-train-val  
dataset: https://www.kaggle.com/bamps53/nfl2021-coco-train-val-annotations  

Now it's way easy to train your own detector.  
In this notebook I'll introduce one of the most famous object detection library, mmdetection.

## Installation

In [ ]:
# just following official install instruction.
# https://github.com/open-mmlab/mmdetection/blob/master/docs/get_started.md
!pip install openmim
!mim install mmdet
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -q -e .

## Setting config

What you need to train detector is just to modify config!

In below example I use FasterRCNN as baseline, but you can use any of configs in mmdetection repository.  
model zoo: https://github.com/open-mmlab/mmdetection/blob/master/docs/model_zoo.md  
configs: https://github.com/open-mmlab/mmdetection/tree/master/configs

But there are so many parameters and it's not easy to fing what you need to change for beginners.  
I've annotated where I modified or added in below example, but I'll summarize what you need to do minimally.

### Dataset
1. You need to change num classes to 1.(We only need to detect only Helmet class.)
2. You need to specify image dir and annotation path for train/valid/test set.

### Model
1. You need to change num classes to 1.(We only need to detect only Helmet class.)

### Others
1. You need to set pretrained model path in `load_from` because what we want to do here is finetune, not training from scratch.
2. Also it's better to decrease LR for finetuning.
3. You can also change `max_epochs` as you want, here I set to 1 for just demonstration.

Note that original config files use inheritation system, but here I included full config in a single file.  
(In my opinion it's easier to understand diff, but may not for others.)

In [ ]:
%%writefile configs/faster_rcnn/custom_faster_rcnn_r50_fpn.py

# dataset settings
dataset_type = 'CocoDataset'
classes = ('Helmet',) # Added
data_root = '/kaggle/input/' # Modified
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        ann_file=data_root + 'nfl2021-coco-train-val-annotations/train.json', # Modified
        img_prefix=data_root + 'nfl2021-train-images/train_images/', # Modified
        classes=classes, # Added
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        ann_file=data_root + 'nfl2021-coco-train-val-annotations/valid.json', # Modified
        img_prefix=data_root + 'nfl2021-train-images/train_images/', # Modified
        classes=classes, # Added
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        ann_file=data_root + 'nfl2021-coco-train-val-annotations/valid.json', # Modified
        img_prefix=data_root + 'nfl2021-train-images/train_images/', # Modified
        classes=classes, # Added
        pipeline=test_pipeline))
evaluation = dict(interval=1, metric='bbox')

# model settings
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[.0, .0, .0, .0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    roi_head=dict(
        type='StandardRoIHead',
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=0),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=dict(
            type='Shared2FCBBoxHead',
            in_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=1, # Modified
            bbox_coder=dict(
                type='DeltaXYWHBBoxCoder',
                target_means=[0., 0., 0., 0.],
                target_stds=[0.1, 0.1, 0.2, 0.2]),
            reg_class_agnostic=False,
            loss_cls=dict(
                type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0))),
    # model training and testing settings
    train_cfg=dict(
        rpn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.7,
                neg_iou_thr=0.3,
                min_pos_iou=0.3,
                match_low_quality=True,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=256,
                pos_fraction=0.5,
                neg_pos_ub=-1,
                add_gt_as_proposals=False),
            allowed_border=-1,
            pos_weight=-1,
            debug=False),
        rpn_proposal=dict(
            nms_pre=2000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            assigner=dict(
                type='MaxIoUAssigner',
                pos_iou_thr=0.5,
                neg_iou_thr=0.5,
                min_pos_iou=0.5,
                match_low_quality=False,
                ignore_iof_thr=-1),
            sampler=dict(
                type='RandomSampler',
                num=512,
                pos_fraction=0.25,
                neg_pos_ub=-1,
                add_gt_as_proposals=True),
            pos_weight=-1,
            debug=False)),
    test_cfg=dict(
        rpn=dict(
            nms_pre=1000,
            max_per_img=1000,
            nms=dict(type='nms', iou_threshold=0.7),
            min_bbox_size=0),
        rcnn=dict(
            score_thr=0.05,
            nms=dict(type='nms', iou_threshold=0.5),
            max_per_img=100)
        # soft-nms is also supported for rcnn testing
        # e.g., nms=dict(type='soft_nms', iou_threshold=0.5, min_score=0.05)
    ))

# optimizer
optimizer = dict(type='SGD', lr=0.002, momentum=0.9, weight_decay=0.0001) # Modified
optimizer_config = dict(grad_clip=None)
# learning policy
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[8,]) # If you changed max_epochs, it's better to change when you decrease LR too here.
runner = dict(type='EpochBasedRunner', max_epochs=10) # You can set max_epochs as you want.

checkpoint_config = dict(interval=1)
# yapf:disable
log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook'),
        # dict(type='TensorboardLoggerHook')
    ])
# yapf:enable
custom_hooks = [dict(type='NumClassCheckHook')]

dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_mstrain_3x_coco/faster_rcnn_r50_fpn_mstrain_3x_coco_20210524_110822-e10bd31c.pth'  # Modified
resume_from = None
workflow = [('train', 1)]


In [ ]:
# training is just one line. 
# !python tools/train.py configs/faster_rcnn/custom_faster_rcnn_r50_fpn.py

# You can also test the model like this.
# !python tools/test.py configs/faster_rcnn/custom_faster_rcnn_r50_fpn.py YOUR_MODEL_PATH --eval bbox